In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import os
import json
from tqdm.notebook import tqdm

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

C:\Users\User\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
with open('config/config.json', 'r') as file:
    config = json.load(file)

# Loading Data

In [3]:
df = pd.read_csv(config['ticker_data_close'], index_col=0)
df_vol = pd.read_csv(config['ticker_data_volume'], index_col=0)
df_index = pd.read_csv(config['ticker_data_sp500'], index_col=0)
df.head()

,SPGI,MCO,CPRT,EFX,FLT,CRL,OMC,IPG,RHI,NLSN,...,ETR,CMS,CNP,AES,EVRG,LNT,ATO,NI,NRG,PNW
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,168.119995,146.130005,43.599998,119.540001,193.869995,110.650002,72.769997,20.250000,55.910000,36.660000,...,82.599998,46.950001,28.020000,10.88,53.139999,42.110001,85.040001,25.360001,28.900000,83.889999
2018-01-03,170.820007,148.860001,43.389999,120.160004,194.960007,110.309998,70.360001,19.760000,55.810001,35.990002,...,81.690002,46.660000,27.959999,10.87,52.320000,41.740002,84.339996,25.200001,28.879999,83.120003
2018-01-04,173.380005,151.600006,43.740002,121.639999,195.460007,109.470001,71.059998,20.170000,55.980000,35.790001,...,80.540001,46.139999,27.990000,10.83,52.099998,41.250000,83.980003,25.090000,28.549999,82.510002
2018-01-05,175.699997,154.119995,43.529999,122.830002,197.000000,111.879997,72.169998,20.190001,56.310001,36.000000,...,79.919998,45.810001,27.870001,10.87,51.660000,41.080002,83.190002,24.790001,28.730000,82.419998
2018-01-08,177.179993,155.070007,43.549999,121.989998,197.160004,111.889999,72.410004,20.350000,56.770000,36.160000,...,80.839996,46.340000,28.040001,10.87,51.720001,41.540001,83.449997,25.000000,29.170000,83.050003


In [4]:
df_na = df.isna().sum() 
stocks_to_drop = df_na[df_na > 0].index.tolist()
df = df.drop(stocks_to_drop, axis=1)

# Preprocessing

In [5]:
df = df.pct_change()[1:]
df = df.T
df.head()

Date,2018-01-03,2018-01-04,2018-01-05,2018-01-08,2018-01-09,2018-01-10,2018-01-11,2018-01-12,2018-01-16,2018-01-17,...,2022-01-19,2022-01-20,2022-01-21,2022-01-24,2022-01-25,2022-01-26,2022-01-27,2022-01-28,2022-01-31,2022-02-01
SPGI,0.016060,0.014987,0.013381,0.008423,0.000339,-0.009197,0.004499,0.007030,-0.010189,0.005460,...,-0.002829,-0.011751,-0.002512,0.002447,-0.044072,-0.004981,-0.006188,0.029209,0.021150,0.005395
MCO,0.018682,0.018407,0.016623,0.006164,0.006771,-0.008391,0.005878,0.009825,-0.010938,0.012088,...,0.006417,-0.003924,-0.009847,0.006464,-0.055365,-0.001508,0.004252,0.032493,0.019286,0.002624
CPRT,-0.004816,0.008066,-0.004801,0.000459,-0.001607,0.000920,0.022518,0.007865,-0.011817,0.000226,...,-0.007823,-0.026535,-0.013318,0.021549,-0.018003,-0.030687,-0.013637,0.032178,0.030537,0.001393
EFX,0.005187,0.012317,0.009783,-0.006839,0.002131,-0.008753,0.002558,0.011194,-0.005617,0.008677,...,-0.028006,-0.022529,-0.013077,0.037800,-0.034715,0.002433,-0.007458,0.040592,0.024396,0.010260
FLT,0.005622,0.002565,0.007879,0.000812,-0.001268,0.020314,0.008312,0.003406,-0.007232,0.007929,...,-0.002687,-0.016794,-0.030738,-0.004240,0.001419,-0.008726,-0.003173,0.036846,0.030046,0.010199


# Feature engineering

In [6]:
config

{'start_date': '2018-10-23',
 'end_date': '2022-10-23',
 'tickers_sectors_path': 'data/ticker_sectors.csv',
 'ticker_data_close': 'data/ticker_data_Close.csv',
 'ticker_data_sp500': 'data/ticker_data_SP500.csv',
 'ticker_data_volume': 'data/ticker_data_Volume.csv',
 'features_path': 'data/features.csv',
 'metrics_path': 'results/metrics_table.csv',
 'predictions_path': 'results/prediction_table.csv'}

In [7]:
df_sectors = pd.read_csv(config['tickers_sectors_path'], index_col=0)
df_sectors.head()

,ticker,sector
0,SPGI,Financial
1,MCO,Financial
2,CPRT,Industrials
3,EFX,Industrials
4,FLT,Technology


In [8]:
dict_tick_sect = dict(zip(df_sectors['ticker'].values.tolist(),
                         df_sectors['sector'].values.tolist()))

df['sector'] = df.index.map(dict_tick_sect)
df.head()

Date,2018-01-03,2018-01-04,2018-01-05,2018-01-08,2018-01-09,2018-01-10,2018-01-11,2018-01-12,2018-01-16,2018-01-17,...,2022-01-20,2022-01-21,2022-01-24,2022-01-25,2022-01-26,2022-01-27,2022-01-28,2022-01-31,2022-02-01,sector
SPGI,0.016060,0.014987,0.013381,0.008423,0.000339,-0.009197,0.004499,0.007030,-0.010189,0.005460,...,-0.011751,-0.002512,0.002447,-0.044072,-0.004981,-0.006188,0.029209,0.021150,0.005395,Financial
MCO,0.018682,0.018407,0.016623,0.006164,0.006771,-0.008391,0.005878,0.009825,-0.010938,0.012088,...,-0.003924,-0.009847,0.006464,-0.055365,-0.001508,0.004252,0.032493,0.019286,0.002624,Financial
CPRT,-0.004816,0.008066,-0.004801,0.000459,-0.001607,0.000920,0.022518,0.007865,-0.011817,0.000226,...,-0.026535,-0.013318,0.021549,-0.018003,-0.030687,-0.013637,0.032178,0.030537,0.001393,Industrials
EFX,0.005187,0.012317,0.009783,-0.006839,0.002131,-0.008753,0.002558,0.011194,-0.005617,0.008677,...,-0.022529,-0.013077,0.037800,-0.034715,0.002433,-0.007458,0.040592,0.024396,0.010260,Industrials
FLT,0.005622,0.002565,0.007879,0.000812,-0.001268,0.020314,0.008312,0.003406,-0.007232,0.007929,...,-0.016794,-0.030738,-0.004240,0.001419,-0.008726,-0.003173,0.036846,0.030046,0.010199,Technology


## calculating features for tables

In [9]:
def calc_growth(prices):
    """
    Calculates list with growth
    """
    growth = []
    past_p = 0
    for p in prices:
        if past_p:
            growth.append(p - past_p)
        past_p = p
    return growth

def find_max_recovery(prices):
    """
    Takes Series with closing prices.
    Returns the value of maximum recovery
    period in days and indexes of prices
    where this recovery period took place.
    """
    growth = calc_growth(prices)
    s = 0
    left = 0
    right = 0
    curr_left = 0
    max_recovery = 0
    for i in range(0, len(growth)):
        if not s:
            curr_left = i
        s += growth[i]
        if s > 0:
          s = 0
          if max_recovery < (i - curr_left):
              max_recovery = i - curr_left
              left = curr_left
              right = i
            
    return max_recovery, left, right + 1

def find_max_drawdown(prices):
    """
    Takes Series with closing prices.
    Returns the value of maximum drawdown
    in percent and indexes of prices where this
    maximum drawdown took place. If stock is
    always growing it will return minimum
    growth with and indexes of prices where this
    minimum growth took place.
    """
    max_price = prices.iloc[0]
    curr_drawdown = 0
    max_drawdown = 0
    curr_left = 0
    left = 0
    right = 0
    for i in range(0, len(prices)):
        curr_drawdown = (prices.iloc[i] / max_price - 1) * 100
        if curr_drawdown < max_drawdown:
            max_drawdown = curr_drawdown
            left = curr_left
            right = i
        if prices.iloc[i] > max_price:
            max_price = prices.iloc[i]
            curr_left = i
    return max_drawdown, left, right


#расчет беты, корреляции, alfa, VAR,

def beta(stock, index):
    stock
    return (prices.cov()['market'] / df_pct.var()['market'])

def alfa_cal(df_pct):
    return df_pct.drop(['market'], axis = 1) - beta_cal(df_pct) * df_pct['market']

def VaR_cal(df_pct, alpha=5):  #historical
#Output the percentile of the distribution at the given alpha confidence level
    return df_pct.drop(['market'], axis = 1).quantile(0.05)

def CVaR_cal(df_pct, alpha=5):
    belowVaR = df_pct.drop(['market'], axis = 1) <= VaR_cal(df_pct, alpha = alpha)
    return df_pct[belowVaR].mean()

In [10]:
df_index = df_index.pct_change()[1:].T
df_index.index = ['market']
df_vol = df_vol.T

df_no_sector = df.drop(['sector'], axis=1)

df_with_market = pd.concat([df_no_sector, df_index])
df_with_market = df_with_market.fillna(0)

table_features = pd.DataFrame(index = df_no_sector.index)

In [11]:
table_features['mean_return'] = df_no_sector.T.mean()
table_features['std_return'] = df_no_sector.T.std()
table_features['median_return'] = df_no_sector.T.median()
table_features['share_positive_return'] = (df_no_sector.T > 0).sum() / df_no_sector.shape[1]

list_max_rec_per = []
list_drawdown = []
list_beta = []
list_alpha = []
list_sharp = []
list_VaR = []
list_CVaR = []
list_CAPM = []
list_coef_var = []
list_IR = []

riskless_return = 0.03 / 252
index = df_with_market.loc['market'].T.values
r_market = np.mean(index)

for ticker in tqdm(df_no_sector.index):
    price = df_no_sector.loc[ticker].T.values
    vol = df_vol.loc[ticker].T.values
    price_cumprod = (df_no_sector.loc[ticker]+1).cumprod()

    time_series = df_no_sector.loc[ticker].T.values
    max_rec_per = find_max_recovery(price_cumprod)[0]
    max_drawdown = find_max_drawdown(price_cumprod)[0]
    
    covar = np.cov(price, index)[0, 1]
    std = table_features.loc[ticker, 'std_return']
    var = std**2
    var_market = np.var(index)
    mean_return = table_features.loc[ticker, 'mean_return']
    
    beta = covar/var_market
    alpha = mean_return - beta * r_market 
    sharp = (mean_return - riskless_return)/std
    VaR = np.quantile(price, 0.05)
    CVaR = price[price < VaR].mean()
    CAPM = riskless_return + beta * (r_market - riskless_return)
    coef_variation = var/mean_return
    IR = (mean_return - r_market) / np.std(price - index)
     
    list_max_rec_per.append(max_rec_per)
    list_drawdown.append(max_drawdown)
    list_beta.append(beta)
    list_alpha.append(alpha)
    list_sharp.append(sharp)
    list_VaR.append(VaR)
    list_CVaR.append(CVaR)
    list_CAPM.append(CAPM)
    list_coef_var.append(coef_variation)
    list_IR.append(IR)

# table_features['beta'] = 
table_features['beta'] = list_beta
table_features['alpha'] = list_alpha
table_features['max_drawdown'] = list_drawdown
table_features['rec_period'] = list_max_rec_per
table_features['sharp'] = list_sharp
table_features['VaR'] = list_VaR
table_features['CAPM'] = list_CAPM
table_features['coef_var'] = list_coef_var
table_features['IR'] = list_IR


  0%|          | 0/487 [00:00<?, ?it/s]

In [12]:
table_features

,mean_return,std_return,median_return,share_positive_return,beta,alpha,max_drawdown,rec_period,sharp,VaR,CAPM,coef_var,IR
SPGI,0.001062,0.018787,0.001406,0.549611,1.090733,0.000410,-38.279342,176,0.050203,-0.024380,0.000642,0.332270,0.038857
MCO,0.001033,0.019999,0.001830,0.563230,1.209063,0.000310,-42.136624,174,0.045692,-0.026926,0.000698,0.387245,0.035792
CPRT,0.001252,0.019626,0.001621,0.558366,1.013998,0.000646,-43.751197,153,0.057741,-0.027040,0.000605,0.307581,0.046008
EFX,0.000874,0.019331,0.001217,0.547665,0.876425,0.000350,-35.800197,197,0.039055,-0.028279,0.000539,0.427563,0.017819
FLT,0.000445,0.021736,0.000662,0.531128,1.107685,-0.000217,-47.781664,98,0.015011,-0.032358,0.000650,1.060916,-0.009557
...,...,...,...,...,...,...,...,...,...,...,...,...,...
LNT,0.000438,0.015180,0.000793,0.524319,0.609427,0.000073,-33.538303,369,0.020990,-0.020072,0.000411,0.526482,-0.011604
ATO,0.000345,0.016044,0.000612,0.525292,0.647978,-0.000042,-33.233806,76,0.014092,-0.021309,0.000430,0.745795,-0.017690
NI,0.000279,0.017453,0.000866,0.525292,0.715286,-0.000149,-31.740835,128,0.009137,-0.021838,0.000462,1.093660,-0.021179
NRG,0.000528,0.021968,0.001164,0.528210,0.949257,-0.000039,-49.296748,498,0.018639,-0.029922,0.000574,0.913122,-0.003883


In [13]:
table_features.describe()

,mean_return,std_return,median_return,share_positive_return,beta,alpha,max_drawdown,rec_period,sharp,VaR,CAPM,coef_var,IR
count,487.000000,487.000000,487.000000,487.000000,487.000000,487.000000,487.000000,487.000000,487.000000,487.000000,487.000000,487.000000,487.000000
mean,0.000683,0.022408,0.000949,0.526305,1.025997,0.000069,-48.419009,326.336756,0.025561,-0.031399,0.000611,-0.595485,0.004411
std,0.000522,0.006104,0.000645,0.020293,0.277415,0.000505,15.364710,233.383472,0.019984,0.007983,0.000133,25.150080,0.025295
min,-0.000695,0.012204,-0.001361,0.371595,0.273979,-0.001279,-91.888087,0.000000,-0.032074,-0.066860,0.000250,-436.717463,-0.067191
25%,0.000372,0.018230,0.000569,0.513619,0.838735,-0.000192,-57.066475,150.500000,0.012723,-0.035590,0.000521,0.423743,-0.013395
50%,0.000622,0.020867,0.000956,0.526265,1.021391,0.000047,-45.445397,276.000000,0.023422,-0.029756,0.000608,0.658423,0.001429
75%,0.000904,0.025138,0.001315,0.539883,1.199313,0.000308,-36.770258,462.000000,0.038799,-0.026068,0.000694,1.089665,0.019780
max,0.005230,0.052197,0.003876,0.582685,2.006925,0.004357,-19.816335,1016.000000,0.097913,-0.018099,0.001081,86.808848,0.094926


In [14]:
table_features.to_csv(config['features_path'])

# Clustering

In [15]:
modelname = 'namepreprocessing_|_surnamepreprocessing_|_parameterspreproc_|_clustmodelname_|_parsmetrsclustering'
parametrs = 'par1name:par1value_par2name:par2value'

example = 'row_|_originalnowindows_|__|_random_|_random_state:42'

In [16]:
n_clusters_ = df_sectors['sector'].nunique()

df_predictions = pd.DataFrame(df['sector'].values, index=df.index, columns=['original'])
df_predictions['original_n'] = LabelEncoder().fit_transform(df_predictions['original'])

dict_features = dict()

df_predictions

,original,original_n
SPGI,Financial,5
MCO,Financial,5
CPRT,Industrials,7
EFX,Industrials,7
FLT,Technology,9
...,...,...
LNT,Utilities,10
ATO,Utilities,10
NI,Utilities,10
NRG,Utilities,10


In [17]:
model_name = 'row_|_originalnowindows_|__|_original_|_'

X = df.drop(['sector'], axis=1).values
dict_features[model_name] = X

clust_pred = df_predictions['original_n']
df_predictions[model_name] = clust_pred

In [ ]:
from sklearn.metrics import silhouette_score

def score_func(estimator, X, y):
    labels_predicted = estimator.fit_predict(X)
    score = silhouette_score(X, labels_predicted)
    return score


params = {"n_clusters": [9, 10, 11, 12, 13]}

X = df.drop(['sector'], axis=1).values
y = df['sector'].values

def gridsearch(model, parameters:dict):
    
    custom_cv=[(slice(None), slice(None))]
    grid_search = GridSearchCV(AgglomerativeClustering(), param_grid=params, 
                               scoring=score_func, cv=custom_cv)
    grid_search.fit(X, y)
    print(grid_search.cv_results_)
    print(grid_search.best_estimator_)
    
    return grid_search.best_estimator_.dict

In [18]:
model_name = 'Kmeans_original'
model_name = 'row_|_originalnowindows_|__|_Kmeans_|_n_clusters:11'

X = df.drop(['sector'], axis=1).values
dict_features[model_name] = X

kmeans = KMeans(n_clusters=n_clusters_, random_state=0).fit(X)
clust_pred = kmeans.labels_
df_predictions[model_name] = clust_pred

In [19]:
model_name = 'row_|_originalnowindows_|__|_random_|_random_state:42'

X = df.drop(['sector'], axis=1).values
dict_features[model_name] = X

np.random.seed(42)
clust_pred = np.random.choice(df_predictions['original_n'].unique(), size=len(df))
df_predictions[model_name] = clust_pred

# Calculating metrics

In [20]:
from sklearn.metrics import (davies_bouldin_score, 
                            silhouette_score,
                            calinski_harabasz_score,
                            homogeneity_score)



metrics = {'silhouette':silhouette_score, 
           'davies_bouldin':davies_bouldin_score, 
           'calinski_harabasz':calinski_harabasz_score, 
           'homogeneity':homogeneity_score}


metrics_df = pd.DataFrame(columns = list(metrics.keys()))



for model in dict_features.keys():
    metrics_list = []
    for metric_name, metric_formula in metrics.items():
        if metric_name == 'homogeneity':
            metric_meaning = metric_formula(df_predictions['original_n'], df_predictions[model])
        else:
            metric_meaning = metric_formula(dict_features[model], df_predictions[model])
        metrics_list.append(metric_meaning)
    metrics_df.loc[model] = metrics_list
    
metrics_df

,silhouette,davies_bouldin,calinski_harabasz,homogeneity
row_|_originalnowindows_|__|_original_|_,-0.014835,3.730855,9.909445,1.000000
row_|_originalnowindows_|__|_Kmeans_|_n_clusters:11,0.017380,2.878425,18.539991,0.375254
row_|_originalnowindows_|__|_random_|_random_state:42,-0.037978,10.148260,0.986009,0.050600


In [23]:
def rank_methods(df):
    
    for c in df.columns:
        if c == 'davies_bouldin':
            df[c] = df[c].rank(method='dense',ascending=True).astype(int)
        else:
            df[c] = df[c].rank(method='dense',ascending=False).astype(int)
            
    return df.mean(axis=1)

def choose_method(df_mean_ranked):
    method = df_mean_ranked.argmin()
    return method

df_mean_ranked = rank_methods(metrics_df.copy()) 
metrics_df['mean_ranks'] = df_mean_ranked
method = choose_method(df_mean_ranked)
print('best_method: '+ df_mean_ranked.index[method])

metrics_df

best_method: row_|_originalnowindows_|__|_Kmeans_|_n_clusters:11


,silhouette,davies_bouldin,calinski_harabasz,homogeneity,sum_ranks,mean_ranks
row_|_originalnowindows_|__|_original_|_,-0.014835,3.730855,9.909445,1.000000,1.75,1.8
row_|_originalnowindows_|__|_Kmeans_|_n_clusters:11,0.017380,2.878425,18.539991,0.375254,1.25,1.6
row_|_originalnowindows_|__|_random_|_random_state:42,-0.037978,10.148260,0.986009,0.050600,3.00,2.6


# Saving results

In [22]:
# metrics_df.to_csv(config['metrics_path'])
# df_predictions.to_csv(config['predictions_path'])